In [2]:
# Import necessary modules
from itertools import combinations

# Function to write CNF clauses to a file in DIMACS format
def write_cnf(clauses, num_vars, filename):
    with open(filename, 'w') as f:
        f.write(f'p cnf {num_vars} {len(clauses)}\n')
        for clause in clauses:
            clause_str = ' '.join(map(str, clause)) + ' 0\n'
            f.write(clause_str)
    print(f"CNF written to {filename}")

# N-Queens problem encoding
def n_queens(n):
    clauses = []
    num_vars = n * n  # Each cell is a variable

    # Mapping from (row, col) to variable number
    var = lambda i, k: i * n + k + 1  # 0-based indexing

    # Each row must have exactly one queen
    for i in range(n):
        # At least one queen in each row
        clauses.append([var(i, k) for k in range(n)])
        # At most one queen in each row
        for k1 in range(n):
            for k2 in range(k1 + 1, n):
                clauses.append([-var(i, k1), -var(i, k2)])

    # Each column must have at most one queen
    for k in range(n):
        for i1 in range(n):
            for i2 in range(i1 + 1, n):
                clauses.append([-var(i1, k), -var(i2, k)])

    # Diagonal constraints
    for i1 in range(n):
        for k1 in range(n):
            for i2 in range(i1 + 1, n):
                # Calculate positions to check for diagonals
                k2_main = k1 + (i2 - i1)
                if k2_main < n:
                    clauses.append([-var(i1, k1), -var(i2, k2_main)])
                k2_anti = k1 - (i2 - i1)
                if k2_anti >= 0:
                    clauses.append([-var(i1, k1), -var(i2, k2_anti)])

    return clauses, num_vars

# All-Interval Series problem encoding
def all_interval(n):
    clauses = []
    num_vars = n * n + (n - 1) * (n - 1)  # Variables for x_i and d_i
    offset = n * n  # Offset for d_i variables

    # Mapping for x_i variables
    x_var = lambda i, k: i * n + k + 1  # i from 0 to n-1, k from 0 to n-1

    # Mapping for d_i variables
    d_var = lambda i, d: offset + i * (n - 1) + d - 1 + 1  # d from 1 to n-1

    # Variables x_i form a permutation of 1..n
    for i in range(n):
        # Each x_i takes exactly one value
        clauses.append([x_var(i, k) for k in range(n)])
        for k1 in range(n):
            for k2 in range(k1 + 1, n):
                clauses.append([-x_var(i, k1), -x_var(i, k2)])

    # All x_i are different
    for k in range(n):
        for i1 in range(n):
            for i2 in range(i1 + 1, n):
                clauses.append([-x_var(i1, k), -x_var(i2, k)])

    # Variables d_i take values from 1 to n-1 and are all different
    for i in range(n - 1):
        clauses.append([d_var(i, d) for d in range(1, n)])
        for d1 in range(1, n):
            for d2 in range(d1 + 1, n):
                clauses.append([-d_var(i, d1), -d_var(i, d2)])

    # All d_i are different
    for d in range(1, n):
        for i1 in range(n - 1):
            for i2 in range(i1 + 1, n - 1):
                clauses.append([-d_var(i1, d), -d_var(i2, d)])

    # Link x_i and d_i variables
    for i in range(n - 1):
        for k1 in range(n):
            for k2 in range(n):
                d = abs(k1 - k2) + 1
                if d >= 1 and d <= n - 1:
                    clauses.append([-x_var(i, k1), -x_var(i + 1, k2), d_var(i, d)])
                else:
                    clauses.append([-x_var(i, k1), -x_var(i + 1, k2)])

    return clauses, num_vars

# 2-MOLS problem encoding
def mols(n):
    clauses = []
    num_vars = 2 * n * n * n  # Variables for x_{i,j,k} and y_{i,j,k}

    # Mapping for x_{i,j,k}
    x_var = lambda i, j, k: i * n * n + j * n + k + 1  # 0-based indexing

    # Mapping for y_{i,j,k}
    y_offset = n * n * n
    y_var = lambda i, j, k: y_offset + i * n * n + j * n + k + 1

    # x and y variables take values from 1 to n
    for i in range(n):
        for j in range(n):
            # Exactly one value for x_{i,j}
            clauses.append([x_var(i, j, k) for k in range(n)])
            for k1 in range(n):
                for k2 in range(k1 + 1, n):
                    clauses.append([-x_var(i, j, k1), -x_var(i, j, k2)])
            # Exactly one value for y_{i,j}
            clauses.append([y_var(i, j, k) for k in range(n)])
            for k1 in range(n):
                for k2 in range(k1 + 1, n):
                    clauses.append([-y_var(i, j, k1), -y_var(i, j, k2)])

    # Latin square constraints for x variables (rows and columns)
    for k in range(n):
        for i in range(n):
            # Rows
            for j1 in range(n):
                for j2 in range(j1 + 1, n):
                    clauses.append([-x_var(i, j1, k), -x_var(i, j2, k)])
            # Columns
            for i1 in range(n):
                for i2 in range(i1 + 1, n):
                    clauses.append([-x_var(i1, k, k), -x_var(i2, k, k)])
    # Same for y variables
    for k in range(n):
        for i in range(n):
            # Rows
            for j1 in range(n):
                for j2 in range(j1 + 1, n):
                    clauses.append([-y_var(i, j1, k), -y_var(i, j2, k)])
            # Columns
            for i1 in range(n):
                for i2 in range(i1 + 1, n):
                    clauses.append([-y_var(i1, k, k), -y_var(i2, k, k)])

    # Mutual orthogonality constraints
    # Map pairs (k,l) to unique values p
    def pair_to_p(k, l):
        return k * n + l

    # For each cell (i,j), define z_{i,j,p}
    z_offset = 2 * n * n * n
    z_var = lambda i, j, p: z_offset + i * n * n + j * n + p + 1

    # Update num_vars to include z variables
    num_vars = z_var(n - 1, n - 1, n * n - 1)

    # Define z_{i,j,p} variables and their constraints
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    p = pair_to_p(k, l)
                    # z_{i,j,p} ↔ (x_{i,j,k} ∧ y_{i,j,l})
                    clauses.append([-z_var(i, j, p), x_var(i, j, k)])
                    clauses.append([-z_var(i, j, p), y_var(i, j, l)])
                    clauses.append([z_var(i, j, p), -x_var(i, j, k), -y_var(i, j, l)])

    # All z_{i,j,p} are different
    for p in range(n * n):
        z_cells = [z_var(i, j, p) for i in range(n) for j in range(n)]
        # At least one z_{i,j,p} is True
        clauses.append(z_cells)
        # At most one z_{i,j,p} is True
        for idx1 in range(len(z_cells)):
            for idx2 in range(idx1 + 1, len(z_cells)):
                clauses.append([-z_cells[idx1], -z_cells[idx2]])

    return clauses, num_vars

# Function to generate and write the CNF based on user input
def generate_cnf(problem_type, n):
    if problem_type == 'n_queens':
        clauses, num_vars = n_queens(n)
        filename = f'../benchmarks_sat/n_queens_{n}.cnf'
    elif problem_type == 'all_interval':
        clauses, num_vars = all_interval(n)
        filename = f'../benchmarks_sat/all_interval_{n}.cnf'
    elif problem_type == 'mols':
        clauses, num_vars = mols(n)
        filename = f'../benchmarks_sat/mols_{n}.cnf'
    else:
        print("Unknown problem type. Use 'n_queens', 'all_interval', or 'mols'.")
        return
    # print(filename)
    write_cnf(clauses, num_vars, filename)

In [ ]:
problem_type = 'n_queens'  # or 'all_interval', 'mols'
for n in range(500, 6001, 500):
    generate_cnf(problem_type, n)

CNF written to ../benchmarks_sat/n_queens_500.cnf


In [3]:
problem_type = 'all_interval'  # or 'all_interval', 'mols'
for n in range(12, 27, 2):
    generate_cnf(problem_type, n)

CNF written to ../benchmarks_sat/all_interval_12.cnf
CNF written to ../benchmarks_sat/all_interval_14.cnf
CNF written to ../benchmarks_sat/all_interval_16.cnf
CNF written to ../benchmarks_sat/all_interval_18.cnf
CNF written to ../benchmarks_sat/all_interval_20.cnf
CNF written to ../benchmarks_sat/all_interval_22.cnf
CNF written to ../benchmarks_sat/all_interval_24.cnf
CNF written to ../benchmarks_sat/all_interval_26.cnf


In [4]:
problem_type = 'mols'  # or 'all_interval', 'mols'
for n in range(3, 10):
    generate_cnf(problem_type, n)

CNF written to ../benchmarks_sat/mols_3.cnf
CNF written to ../benchmarks_sat/mols_4.cnf
CNF written to ../benchmarks_sat/mols_5.cnf
CNF written to ../benchmarks_sat/mols_6.cnf
CNF written to ../benchmarks_sat/mols_7.cnf
CNF written to ../benchmarks_sat/mols_8.cnf
CNF written to ../benchmarks_sat/mols_9.cnf
